In [33]:
import os
import pandas as pd

In [34]:
os.chdir('C:\Springboard\Github\Ch26_2_Relax')

In [35]:
# import the user file
df1 = pd.read_csv(r'takehome_users.csv', engine='python')
df1.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [36]:
df1.shape

(12000, 10)

In [37]:
# import the user data file
df2 = pd.read_csv(r'takehome_user_engagement.csv', engine='python')
df2.head()

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


In [47]:
df2.shape

(207917, 3)

before merging these DF's, why don't we pair down this df2 to just the information that we need:
If a particular user has logged in more than 3 days in a 7 day period

In [43]:
# how do I do this?
df3 = df2.melt(id_vars=['user_id'], value_vars=['time_stamp'])
df3

,user_id,variable,value
0,1,time_stamp,2014-04-22 03:53:30
1,2,time_stamp,2013-11-15 03:45:04
2,2,time_stamp,2013-11-29 03:45:04
3,2,time_stamp,2013-12-09 03:45:04
4,2,time_stamp,2013-12-25 03:45:04
...,...,...,...
207912,11996,time_stamp,2013-09-06 06:14:15
207913,11997,time_stamp,2013-01-15 18:28:37
207914,11998,time_stamp,2014-04-27 12:45:16
207915,11999,time_stamp,2012-06-02 11:55:59


In [52]:
df_repeat = df3.groupby('user_id').count()
df_repeat

,variable,value
user_id,,
1,1,1
2,14,14
3,1,1
4,1,1
5,1,1
...,...,...
11996,1,1
11997,1,1
11998,1,1


In [56]:
df_repeat = df_repeat[df_repeat['variable'] > 2]
df_repeat.index

Int64Index([    2,    10,    20,    33,    42,    43,    50,    53,    59,
               60,
            ...
            11961, 11964, 11965, 11967, 11969, 11975, 11980, 11981, 11988,
            11991],
           dtype='int64', name='user_id', length=2248)

That gives us a list of the user that have logged in more than 3 times total, but now we need to narrow that down to the users that have logged in 3 times in the same week. How do I do that? <br>
Let's apply that list to the dataframe of logins to start

In [57]:
df2 = df2[df2['user_id'].isin(df_repeat.index)]
df2

,time_stamp,user_id,visited
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1
5,2013-12-31 03:45:04,2,1
...,...,...,...
207905,2014-04-20 14:22:45,11991,1
207906,2014-04-25 14:22:45,11991,1
207907,2014-04-28 14:22:45,11991,1
207908,2014-05-17 14:22:45,11991,1


That didn't do much, as we only dropped 6,000 rows from the 207,000 that we started with.

# back to the original question: how do we get the user_id as the index and the time_stamp as columns?
# pivot table?
# join the dataframe on itself?


Both DF's are loaded; we need to merge these 2 DF's. <br>
Before we merge, we need to make the columns the same name

In [59]:
df1.rename(columns={'object_id':'user_id'}, inplace=True)

In [60]:
df1.columns

Index(['user_id', 'creation_time', 'name', 'email', 'creation_source',
       'last_session_creation_time', 'opted_in_to_mailing_list',
       'enabled_for_marketing_drip', 'org_id', 'invited_by_user_id'],
      dtype='object')

In [66]:
df = df1.merge(df2, how='left', on='user_id')
df.shape

(210754, 12)

In [67]:
df.head()

,user_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,time_stamp,visited
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0,NaN,NaN
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,2013-11-15 03:45:04,1.0
2,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,2013-11-29 03:45:04,1.0
3,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,2013-12-09 03:45:04,1.0
4,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,2013-12-25 03:45:04,1.0


In [ ]:
for user in df['user_id']:
    temp = df[df['user'] == ]
    for ts in 